In [ ]:
import os
from data_fetcher import YahooFinanceDataFetcher
from mt5_fetcher import MT5DataFetcher
from google_sheet_api import GoogleSheetsUploader
from analyzer import run_analysis, run_all_analyses

# ========== Setup ========== #
BASE_DIR = os.getcwd()
CONFIG_PATH = os.path.join(BASE_DIR, "config.yaml")
CREDENTIAL_PATH = os.path.join(BASE_DIR, "credential_google_sheets.json")
PLOTS_DIR = os.path.join(BASE_DIR, "plots")
os.makedirs(PLOTS_DIR, exist_ok=True)

# ========== Parameters ========== #
fetch_mt5 = True
STD_MULTIPLIER = 1.96
initial_balance = 1000
invest_per_week = 100
tp_percent = 1.0
leverage = 1000
min_years_required = 1

def main():
    # ========== Fetch All Data ========== #
    fetcher = YahooFinanceDataFetcher(CONFIG_PATH)
    full_df = fetcher.get_data()
    symbol_list = full_df["Symbol"].unique()

    # ========== Run Single Symbol Analysis ========== #
    symbol = "GOOGL.NAS"
    data, dd_thresh, gain_thresh, df = run_analysis(
        df=full_df,
        symbol=symbol,
        std_multiplier=STD_MULTIPLIER,
        plots_dir=PLOTS_DIR,
        tp_percent=tp_percent,
        leverage=leverage,
        coeff=fetcher.coeff_map.get(symbol),
        initial_balance=initial_balance,
        invest_per_week=invest_per_week,
        min_years_required=min_years_required
    )
    
    # ========== Run All Analyses ========== #
    df = run_all_analyses(
        full_df=full_df,
        symbol_list=symbol_list,
        std_multiplier=STD_MULTIPLIER,
        plots_dir=PLOTS_DIR,
        tp_percent=tp_percent,
        leverage=leverage,
        coeff_map=fetcher.coeff_map,
        initial_balance=initial_balance,
        invest_per_week=invest_per_week,
        min_years_required=min_years_required
    )

    # ========== Fetch MT5 Live Prices ========== #
    if fetch_mt5:
        df = MT5DataFetcher(df, min_years_required)

    # ========== Upload to Google Sheets ========== #
    try:
        print("\n📤 Uploading to Google Sheets...")
        uploader = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")
        uploader.upload_dataframe(df, "Overview", replace = False)
        print("✅ Upload successful!")
    except Exception as e:
        print(f"❌ Upload failed: {e}")
    
if __name__ == "__main__":
    main()

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

from data_fetcher import YahooFinanceDataFetcher
from mt5_fetcher import MT5DataFetcher
from google_sheet_api import GoogleSheetsUploader
from analyzer import run_analysis, run_all_analyses

# ========== Configuration & Setup ========== #
BASE_DIR = os.getcwd()
CONFIG_PATH = os.path.join(BASE_DIR, "config.yaml")
CREDENTIAL_PATH = os.path.join(BASE_DIR, "credential_google_sheets.json")
PLOTS_DIR = os.path.join(BASE_DIR, "plots")
os.makedirs(PLOTS_DIR, exist_ok=True)

# ========== Parameters ========== #
fetch_mt5 = True
STD_MULTIPLIER = 1.96
initial_balance = 1000
invest_per_week = 100
tp_percent = 1.0
leverage = 1000
min_years_required = 1

def normalize(row):
    action = str(row["Action"]).lower()
    if action == "buy" and pd.notnull(row["Worst Drawdown (%)"]) and row["Worst Drawdown (%)"] != 0:
        return row["Max_Drawdown"] * 100 / -row["Worst Drawdown (%)"]
    elif action == "sell" and pd.notnull(row["Max Gain (%)"]) and row["Max Gain (%)"] != 0:
        return row["Max_Gain"] * 100 / row["Max Gain (%)"]
    return None

def main():
    # ========== Fetch Data ========== #
    fetcher = YahooFinanceDataFetcher(CONFIG_PATH)
    full_df = fetcher.get_data()
    symbol_list = full_df["Symbol"].unique()

    # Run single symbol analysis (example: GOOGL.NAS)
    symbol = "GOOGL.NAS"
    run_analysis(
        df=full_df,
        symbol=symbol,
        std_multiplier=STD_MULTIPLIER,
        plots_dir=PLOTS_DIR,
        tp_percent=tp_percent,
        leverage=leverage,
        coeff=fetcher.coeff_map.get(symbol),
        initial_balance=initial_balance,
        invest_per_week=invest_per_week,
        min_years_required=min_years_required
    )

    # Run all analyses
    df, df_final = run_all_analyses(
        full_df=full_df,
        symbol_list=symbol_list,
        std_multiplier=STD_MULTIPLIER,
        plots_dir=PLOTS_DIR,
        tp_percent=tp_percent,
        leverage=leverage,
        coeff_map=fetcher.coeff_map,
        initial_balance=initial_balance,
        invest_per_week=invest_per_week,
        min_years_required=min_years_required
    )

    # Optional: Fetch live prices from MT5
    if fetch_mt5:
        df = MT5DataFetcher(df, min_years_required)

    # Fetch Action Data from Google Sheets
    sheet_fetcher = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")
    df_raw = sheet_fetcher.get_sheet_as_dataframe("Forex")
    df_sheet = df_raw.iloc[6:].copy()
    df_sheet.columns = df_raw.iloc[5]
    df_sheet = df_sheet[["Symbol", "Type", "Action", "Max Gain (%)", "Worst Drawdown (%)"]].dropna(subset=["Symbol"]).reset_index(drop=True)

    # Merge Action if missing
    if "Action" not in df_final.columns and "Action" in df_sheet.columns:
        df_final = df_final.merge(df_sheet[["Symbol", "Action"]], on="Symbol", how="left")

    # Merge and Normalize
    df_final = df_final[["Symbol", "Datetime", "Action", "Max_Drawdown", "Max_Gain"]].copy()
    df_info = df_sheet[["Symbol", "Max Gain (%)", "Worst Drawdown (%)"]].drop_duplicates("Symbol")
    df_merged = df_final.merge(df_info, on="Symbol", how="left")
    df_merged["Datetime"] = pd.to_datetime(df_merged["Datetime"])
    df_merged["Normalized"] = df_merged.apply(normalize, axis=1)
    df_merged.dropna(subset=["Normalized"], inplace=True)

    # Weighting: Half weight if both buy/sell exist
    weight_map = (
        df_merged.groupby(["Symbol", "Datetime"])["Action"]
        .nunique()
        .reset_index(name="Action_Count")
    )
    weight_map["Weight"] = weight_map["Action_Count"].apply(lambda x: 0.5 if x > 1 else 1.0)
    df_merged = df_merged.merge(weight_map[["Symbol", "Datetime", "Weight"]], on=["Symbol", "Datetime"], how="left")
    df_merged["Weighted_Normalized"] = df_merged["Normalized"] * df_merged["Weight"]

    # Calculate Average Normalized Value by Date
    avg_per_date = (
        df_merged.groupby("Datetime")["Weighted_Normalized"]
        .mean()
        .reset_index()
        .rename(columns={"Weighted_Normalized": "Avg_Normalized_Value"})
    )

    # Plot
    plt.figure(figsize=(12, 6))
    plt.plot(avg_per_date["Datetime"], avg_per_date["Avg_Normalized_Value"], marker='o', label='Avg Normalized Value')
    q5 = avg_per_date["Avg_Normalized_Value"].quantile(0.05)
    plt.axhline(y=q5, color='red', linestyle='--', label=f'5th Percentile ({q5:.2f}%)')
    plt.title("Average Weighted Normalized Drawdown/Gain Over Time")
    plt.xlabel("Date")
    plt.ylabel("Avg Normalized Value (%)")
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Upload to Google Sheets
    try:
        print("\n📤 Uploading to Google Sheets...")
        uploader = GoogleSheetsUploader(CREDENTIAL_PATH, "Financial Report - Indonesia")
        uploader.upload_dataframe(df, "Overview", replace=False)
        print("✅ Upload successful!")
    except Exception as e:
        print(f"❌ Upload failed: {e}")

    return avg_per_date

if __name__ == "__main__":
    avg_df = main()

📈 Fetching AUDCAD (AUDCAD=X)...
